In [1]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(9, 16)
        #self.fc1.weight.data.fill_(0.0)
        #self.fc1.bias.data.fill_(0.0)
        self.fc2 = nn.Linear(16, 16)
        #self.fc2.weight.data.fill_(0.0)
        #self.fc2.bias.data.fill_(0.0)
        self.fc3 = nn.Linear(16, 1)
        #self.fc3.weight.data.fill_(0.0)
        #self.fc3.bias.data.fill_(0.0)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

In [2]:
def optimize(model, optimizer, x, y):
    y_guess = model(x)
    #print("Y:", y)
    #print("Y guess:", y_guess)
    y_guess = y_guess.clone()#.detach().requires_grad_()
    loss = F.smooth_l1_loss(y_guess, y)
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()    

In [3]:
import torch
import numpy as np

x1 = torch.tensor(np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
]).flatten()).float().requires_grad_()
y1 = torch.tensor(0).float()

x2 = torch.tensor(np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
]).flatten()).float().requires_grad_()
y2 = torch.tensor(1).float()

x3 = torch.tensor(np.array([
    [-1, -1, -1],
    [-1, -1, -1],
    [-1, -1, -1]
]).flatten()).float().requires_grad_()
y3 = torch.tensor(-1).float()

In [4]:
print(x1)
all_xs = torch.cat([x1.unsqueeze(0), x2.unsqueeze(0), x3.unsqueeze(0)])
print(all_xs)
#all_xs = torch.tensor(all_xs)
#all_xs = torch.tensor([x1, x2, x3])
all_ys = torch.tensor([y1, y2, y3]).unsqueeze(1)
print(all_ys)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)
tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [-1., -1., -1., -1., -1., -1., -1., -1., -1.]], grad_fn=<CatBackward>)
tensor([[ 0.],
        [ 1.],
        [-1.]])


In [5]:
from torch import optim
import time

model = Model()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
print(model(x1))
print(model(x2))
print(model(x3))

tensor([0.1645], grad_fn=<AddBackward0>)
tensor([0.1817], grad_fn=<AddBackward0>)
tensor([0.1486], grad_fn=<AddBackward0>)


In [7]:
start = int(round(time.time() * 1000))

for i in range(20_000):
    optimize(model, optimizer, x1, y1)
    optimize(model, optimizer, x2, y2)
    optimize(model, optimizer, x3, y3)
    #optimize(model, optimizer, all_xs, all_ys)

end = int(round(time.time() * 1000))
print("Millisec:", end - start)
print("Sec:", (end - start) / 1000)

c:\users\linus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  


Millisec: 40823
Sec: 40.823


In [8]:
# Batching all at once 13.359 / 13.731 sec
# Batching one after another 40.618 / 40.823 sec

In [9]:
print(model(x1))
print(model(x2))
print(model(x3))

tensor([0.], grad_fn=<AddBackward0>)
tensor([1.], grad_fn=<AddBackward0>)
tensor([-1.], grad_fn=<AddBackward0>)


In [10]:
print(model(all_xs))

tensor([[ 0.],
        [ 1.],
        [-1.]], grad_fn=<AddmmBackward>)
